# Global Model
Here we fit a global model using NeuralProphet. We adopt the ERCOT dataset for a simple example.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes

import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
from neuralprophet import set_random_seed
set_random_seed(8)
set_log_level("ERROR","INFO")

Error: Session cannot generate requests

In [ ]:
# data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
# df_ercot = pd.read_csv(data_location + "energy/ERCOT_load.csv")
data_location = '/mnt/c/Users/Pateu/Neural_Prophet_Project/other_files/'
df_ercot = pd.read_csv(data_location+'ERCOT_load_2004_2021Sept.csv')
df_ercot.head()

In [ ]:
regions = list(df_ercot)[1:-1]
regions

In [ ]:
df_list=list()
for cols in regions:
    aux=df_ercot[['ds',cols]].copy()
    aux=aux.iloc[:26301,:].copy()
    aux=aux.rename(columns={cols:'y'})
    df_list.append(aux)

In [ ]:
df_list

In [ ]:
m=NeuralProphet(n_lags=24)
df_train_list,df_test_list=m.split_df(df_list,freq='H',valid_p=0.33)

In [ ]:
#Normalize each region separately:
df_train_list_n=list()
df_test_list_n=list()
i=1
for df_train,df_test in zip(df_train_list,df_test_list): 
    print(i)
    df_train_n=df_train.copy(deep=True)
    df_test_n=df_test.copy(deep=True)
    norm_parameter=df_train_n['y'].abs().max()
    df_train_n['y']=df_train_n['y']/norm_parameter
    df_test_n['y']=df_test_n['y']/norm_parameter
    df_train_list_n.append(df_train_n)
    df_test_list_n.append(df_test_n)   
    i=i+1

In [ ]:
print(df_train_list[0].head())
print(df_train_list_n[0].head())
print(df_test_list[0].head())
print(df_test_list_n[0].head())

In [ ]:
metrics = m.fit(df_train_list_n, freq='H')
metrics.tail()

In [ ]:
metrics

## Test single region

In [ ]:
region=3
print(f'Testing single region: {regions[region]}')
test_metrics=m.test(df_test_list_n[region])
test_metrics

In [ ]:
future = m.make_future_dataframe(df_test_list_n[region], n_historic_predictions=True)
forecast = m.predict(future)
fig = m.plot(forecast)

## Test all regions

In [ ]:
test_metrics=m.test(df_test_list_n)
test_metrics

In [ ]:
future = m.make_future_dataframe(df_test_list_n, n_historic_predictions=True)
forecast = m.predict(future)
for frst in forecast:
    fig = m.plot(frst)

In [ ]:
# for frst in forecast:
#     fig_comp = m.plot_components(frst)

In [ ]:
fig_param = m.plot_parameters()